<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/CV2_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 08/05/2021-3
from __future__ import absolute_import
from IPython.display import clear_output
import glob, os, shutil, sys
from os.path import *

try:
    from google.colab import drive, files
    from google.colab.patches import cv2_imshow
except Exception as err:
    pass

try: 
    from matplotlib import pyplot as plt
except Exception as err:
    %pip install matplotlib
    from matplotlib import pyplot as plt

try: 
    import cv2
    from cv2 import COLOR_BGR2RGBA
except Exception as err:
    print(err)
    %pip install opencv-python-headless # works.
    # %pip install opencv-contrib-python-headless
    import cv2
    from cv2 import COLOR_BGR2RGBA, IMREAD_UNCHANGED

import numpy as np

if exists('/content/sample_data'):
    shutil.rmtree('/content/sample_data')

contentPth = os.getcwd()
tensorflowImagesPath = join(contentPth, 'TensorflowImages')
resizedImagesPath = join(contentPth, 'TensorflowResizedImages')

if exists(resizedImagesPath):
    pass
    shutil.rmtree(resizedImagesPath)

try:
    drive.flush_and_unmount()
    drive.mount('/content/gdrive/', force_remount=True)
    mydrivePth = '/content/gdrive/MyDrive'
    os.chdir(mydrivePth)
    gdriveTFImagesPth = join(mydrivePth, 'TensorflowImages')
    gdriveResizedImagesPth = join(mydrivePth, 'TensorflowResizedImages')
    
    os.chdir(mydrivePth)
    from TarfileFunctions import *
    from BashColors import C
    
    if not exists(tensorflowImagesPath):
        shutil.copytree(gdriveTFImagesPth, tensorflowImagesPath)
        # tff.listTarfiles()
        os.chdir(contentPth)
        # tff.extractTarfiles('Juno_Original_Images.tar.gz')

    gdrive_resized_images = '/content/gdrive/MyDrive/TensorflowResizedImages'
    if not exists(resizedImagesPath):
        try: shutil.copytree(gdrive_resized_images, resizedImagesPath)
        except Exception as err:
            print(err)

    os.chdir(contentPth)

except Exception as err:
    # print(err)
    os.chdir(contentPth)
    from TarfileFunctions import *
    from BashColors import C
    tarfilePath=join(contentPth, 'Juno_Original_Images.tar.gz')
    if exists(tarfilePath) and not exists(tensorflowImagesPath):
        tff.extractTarfiles('Juno_Original_Images.tar.gz')

rootPth = expanduser("~")

os.chdir(contentPth)
print(f'cv2: {C.IBlue}{cv2.__version__}{C.ColorOff}')
print(f'cwd: {C.IBlue}{os.getcwd()}')

In [ ]:
os.chdir(tensorflowImagesPath)
imageGlob = glob.glob('**', recursive=True)
pathList=[]
count=0
for path in sorted(imageGlob):
    fullPath = abspath(path)
    if isfile(path) and not path.__contains__('backgroundImage224.png'):
        pathList.append(fullPath)
        print(count, fullPath)
        count+=1
clear_output()

In [ ]:
zeroPixel:np.ndarray
originalImageZeroPixel:np.ndarray

def matPlotShowSingleImage(thisImage):
    thisImage = cv2.COLOR_BAYER_BG2RGBA
    fig=plt.figure()
    ax1=fig.add_subplot(1,1,1)
    ax1.imshow(thisImage)
    ax1.axis('off')
    pass

def matPlotCompareTwoImages(thisImage, compareImage):
    thisImage = cv2.cvtColor(thisImage, cv2.COLOR_BGR2BGRA)
    compareImage = cv2.cvtColor(compareImage, cv2.COLOR_BGR2BGRA)
    # Plot the images
    fig=plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(thisImage)
    ax1.axis('off')
    ax1.set_title('Original Image')

    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(compareImage)
    ax2.axis('off')
    ax2.set_title('New Image')
    plt.show()

def cv2ResizeImage(thisImage, newSize=(224,224)):
    resized_image = cv2.resize(thisImage,newSize,interpolation=cv2.INTER_CUBIC)
    return resized_image

def cv2CreateImageWithColor(pxColor=[0,0,0,0], silent=True):
    '''returns save_path'''
    bgImagePath = join(contentPth, 'TensorflowImages/backgroundImage224.png')
    bgImage = cv2.imread(bgImagePath, cv2.IMREAD_UNCHANGED)
    save_path = join(contentPth, 'bgImage.png')
    # print(save_path)
    bgImage = np.copy(bgImage)
    print(bgImage[0][0])


    for px in bgImage:
        bgImage[:] = pxColor
        cv2.waitKey(100)
        
    cv2.imwrite(save_path, bgImage)
    if not silent:
        print(f'shape: {bgImage.shape}')
        # cv2_imshow(bgImage)
        matPlotCompareTwoImages(bgImage, bgImage)
    return save_path

def cv2ZoomImage(thisImage, newScale=1, silent=True):
    ''''''
    zeroPixel = thisImage[0][0]
    angle = 0
    width, height, _ = thisImage.shape
    rotPoint = width//2, height //2
    dimentions = width, height
    rotPoint = width//2, height//2
    rotMatrix = cv2.getRotationMatrix2D(rotPoint, angle, scale=newScale)
    zoomImage = cv2.warpAffine(thisImage, rotMatrix, dimentions)
    matPlotCompareTwoImages(thisImage, zoomImage)
    return zoomImage

def cv2Rotation(thisImage, angle=0, rotPoint=None, newScale=1, silent=True):
    ''''''
    thisImage = np.copy(thisImage)
    zeroPixel = thisImage[[0][0]]
    width, height, _ = thisImage.shape
    if rotPoint == None:
        rotPoint = width//2, height//2
        rotMat = cv2.getRotationMatrix2D(rotPoint, angle, scale=newScale)
        dimentions = width, height
        rotImage = cv2.warpAffine(thisImage, rotMat, dimentions)
        rotImageZeroPixel = rotImage[0][0]
        if not silent:
            print('cv2Rotation()')
            matPlotCompareTwoImages(thisImage, rotImage)
            cv2.waitKey(100)
        cv2.waitKey(100)
        cv2.destroyAllWindows()
        return rotImage

def cv2Translate(thisImage, x=0, y=0, silent=True):
    '''-x shift left -y shift up\nx shift right y shift down'''
    thisImage = np.copy(thisImage)
    zeroPixel=thisImage[0][0]
    translateMatrix = np.float32([[1,0,x],[0,1,y]])
    dimentions = (thisImage.shape[1], thisImage.shape[0])
    newImage = cv2.warpAffine(thisImage, translateMatrix, dimentions)
    if not silent:
        print('cv2Translate()')
        matPlotCompareTwoImages(thisImage, newImage)
        cv2.waitKey(100)
    cv2.waitKey(100)
    cv2.destroyAllWindows()
    return newImage

def cv2EdgeDetection(thisImage, t1=50, t2=50):
    ''''''
    canny = cv2.Canny(thisImage, t1, t2)
    matPlotCompareTwoImages(thisImage, canny)
    cv2.waitKey(100)
    cv2.destroyAllWindows()
    return canny

def changeBackgroundColors(path, image=None):
    '''returns newImg'''
    splitPath = split(path)
    filePath = splitPath[0]
    fileName = splitPath[1]
    fileName = 'new_' + fileName
    print(filePath)
    print(fileName)
    saveImagePath = join(contentPth, fileName)

    print(f'saveImagePath: {saveImagePath}')
    original_image = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    originalZeroPixel = originalImage[0][0]
    plt.imshow(original_image)
    newImg = np.copy(original_image)

    zeroPixel = newImg[0][0]
    print('newImg zeroPixel:', zeroPixel)
    print('originalZeroPixel:', originalZeroPixel)

    width, height, channels = original_image.shape
    print(width, height, channels)

    for x in range(0, width):
        for y in range(0, height):
            channels_xy = newImg[y][x]
            # print(channels_xy)
            if all(channels_xy == zeroPixel):
                newImg[y][x] = originalZeroPixel
                
    cv2.imwrite(saveImagePath, newImg)
    newImg = cv2.imread(saveImagePath, -1)
    return newImg
    # plt.imshow(newImg)


In [ ]:
help(cv2CreateImageWithColor)

In [ ]:
imagePath = pathList[0]
imageName = basename(imagePath)
originalImage=cv2.imread(imagePath, -1)
originalImage = cv2ResizeImage(originalImage)
originalZeroPixel = originalImage[0][0]

img = cv2Rotation(originalImage, 180, newScale=.75, silent=False)
savePath = join(contentPth, imageName)
cv2.imwrite(savePath, img)

newImg = changeBackgroundColors(savePath)
cv2_imshow(newImg)
cv2.waitKey(100)
cv2.destroyAllWindows()
# img = cv2Translate(thisImage=img, x=0 ,y=50, silent=False)
print(originalZeroPixel)

In [ ]:
imgPath = pathList[0]
originalImage = cv2.imread(imgPath, cv2.IMREAD_UNCHANGED)
originalImage = cv2ResizeImage(originalImage)
canny = cv2EdgeDetection(originalImage)
canny = cv2EdgeDetection(originalImage, t1=100, t2=100)
cv2.waitKey(100)
cv2.destroyAllWindows()

In [ ]:
count=0
for pth in sorted(pathList):
    imagePath = pth
    enemyImage = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
    enemyImage = cv2.resize(enemyImage, (224, 224), 
                            interpolation=cv2.INTER_CUBIC)
    print(f'pathList[{count}]')
    count+=1
    print(imagePath)

    zeroPixel = enemyImage[0][0]

    bgImagePath = cv2CreateImageWithColor(pxColor=zeroPixel)
    bgImage = cv2.imread(bgImagePath, cv2.IMREAD_UNCHANGED)
    cv2_imshow(enemyImage)

    imageSecondary = cv2.hconcat([bgImage,bgImage,bgImage])
    imagePrimary = cv2.hconcat([bgImage,enemyImage,bgImage])
    zoomImage = cv2.vconcat([imageSecondary,imagePrimary,imageSecondary])
    zoomImage = cv2ResizeImage(thisImage=zoomImage)

    print(zoomImage.shape)
    cv2_imshow(zoomImage)
    matPlotCompareTwoImages(enemyImage, zoomImage)

    edges = cv2EdgeDetection(zoomImage)
    
    cv2.waitKey(200)
    cv2.destroyAllWindows()
    print()

In [ ]:
_=cv2Rotation(originalImage, silent=False)

In [ ]:
# cv2CreateImageWithColor()
imagePath = cv2CreateImageWithColor(pxColor=[0,0,255,255])
print(imagePath)
img = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
cv2_imshow(img)
# matPlotShowSingleImage(img)

In [ ]:

import cv2
import numpy as np
from time import sleep
imagePath = join(contentPth, 'enemy_1.png')
img = cv2.imread(imagePath , cv2.IMREAD_UNCHANGED)
cv2_imshow(img)
plt.imshow(img)
cv2.waitKey(200)

row,col,plane = img.shape
print(row,col,plane)
# here image is of class 'uint8', the range of values  
# that each color component can have is [0 - 255]

# create a zero matrix of order same as
# original image matrix order of same dimension
temp = np.zeros((row,col,plane),np.uint8)

# store blue plane contents or data of image matrix
# to the corresponding plane(blue) of temp matrix
temp[:,:,0] = img[:,:,0]
planeZERO=temp
# displaying the Blue plane image
cv2_imshow(temp)
print()
# again take a zero matrix of image matrix shape
temp = np.zeros((row,col,plane),np.uint8)

# store green plane contents or data of image matrix
# to the corresponding plane(green) of temp matrix
temp[:,:,1] = img[:,:,1]
planeONE = temp
# displaying the Green plane image
cv2_imshow(temp)
print()
# again take a zero matrix of image matrix shape
temp = np.zeros((row,col,plane),np.uint8)

# store red plane contents or data of image matrix
# to the corresponding plane(red) of temp matrix
temp[:,:,2] = img[:,:,2]
cv2_imshow(temp)
print()
planeTWO = temp
# displaying the Red plane image
cv2_imshow(temp)
matPlotCompareTwoImages(img, img)